In [1]:
import os
import json
import sqlite3
from openai import OpenAI
import logging
from tqdm import tqdm
from dotenv import load_dotenv
from openai import RateLimitError
import backoff

# Load environment variables from .env file
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
@backoff.on_exception(backoff.expo, RateLimitError)

def create_db_tables(conn):
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS completions (
            id INTEGER PRIMARY KEY,
            custom_id TEXT UNIQUE,
            content TEXT
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS checkpoints (
            id INTEGER PRIMARY KEY,
            file_path TEXT UNIQUE,
            last_processed_line INTEGER
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY,
            custom_id TEXT UNIQUE,
            konsert_datum TEXT,
            konsert_namn TEXT,
            lokal_namn TEXT,
            konserttyp_namn TEXT,
            producer TEXT
        )
    ''')
    conn.commit()

def get_checkpoint(conn, file_path):
    cursor = conn.cursor()
    cursor.execute('SELECT last_processed_line FROM checkpoints WHERE file_path = ?', (file_path,))
    result = cursor.fetchone()
    return result[0] if result else 0

def update_checkpoint(conn, file_path, last_processed_line):
    cursor = conn.cursor()
    cursor.execute('''
        INSERT OR REPLACE INTO checkpoints (file_path, last_processed_line)
        VALUES (?, ?)
    ''', (file_path, last_processed_line))
    conn.commit()

def extract_and_store_event_data(cursor, custom_id, json_response):
    try:
        event_data = json.loads(json_response)
        cursor.execute('''
            INSERT OR REPLACE INTO events 
            (custom_id, konsert_datum, konsert_namn, lokal_namn, konserttyp_namn, producer)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            custom_id,
            event_data.get('konsert_datum', ''),
            event_data.get('konsert_namn', ''),
            event_data.get('lokal_namn', ''),
            event_data.get('konserttyp_namn', ''),
            event_data.get('Producer', '')
        ))
    except json.JSONDecodeError:
        logging.error(f"Error decoding JSON for custom_id: {custom_id}")
    except Exception as e:
        logging.error(f"Error storing event data for custom_id {custom_id}: {e}")

def process_jsonl(file_path, db_conn):
    cursor = db_conn.cursor()
    last_processed_line = get_checkpoint(db_conn, file_path)
    
    try:
        with open(file_path, 'r') as file:
            # Skip to the last processed line
            for _ in range(last_processed_line):
                next(file)
            
            # Count remaining lines for progress bar
            remaining_lines = sum(1 for _ in file) - last_processed_line
            file.seek(0, 0)  # Reset file pointer
            for _ in range(last_processed_line):
                next(file)
            
            for current_line, line in tqdm(enumerate(file, start=last_processed_line), total=remaining_lines, desc="Processing lines"):
                try:
                    data = json.loads(line.strip())
                    messages = data['body']['messages']
                    custom_id = data.get('custom_id', f"line_{current_line}")
                    
                    # Check if this custom_id has already been processed
                    cursor.execute('SELECT id FROM completions WHERE custom_id = ?', (custom_id,))
                    if cursor.fetchone():
                        logging.info(f"Skipping already processed custom_id: {custom_id}")
                        continue
                    
                    completion = client.chat.completions.create(
                        model='gpt-3.5-turbo',
                        response_format={"type": "json_object"},
                        messages=messages,
                        max_tokens=data['body']['max_tokens']
                    )
                    
                    json_response = completion.choices[0].message.content
                    
                    # Store the result in the completions table
                    cursor.execute('INSERT INTO completions (custom_id, content) VALUES (?, ?)',
                                   (custom_id, json_response))
                    
                    # Extract and store event data
                    extract_and_store_event_data(cursor, custom_id, json_response)
                    
                    # Update checkpoint every 10 lines
                    if current_line % 10 == 0:
                        update_checkpoint(db_conn, file_path, current_line)
                        db_conn.commit()
                    
                except json.JSONDecodeError:
                    logging.error(f"Error decoding JSON at line {current_line}")
                except Exception as e:
                    logging.error(f"Error during API call at line {current_line}: {e}")
                
            # Final checkpoint update
            update_checkpoint(db_conn, file_path, current_line)
            db_conn.commit()
            
    except IOError as e:
        logging.error(f"Error opening or reading the file: {file_path}. Error: {e}")

def process_all_jsonl_files(directory_path, db_conn):
    jsonl_files = [f for f in os.listdir(directory_path) if f.endswith('.jsonl')]
    for file_name in tqdm(jsonl_files, desc="Processing files"):
        file_path = os.path.join(directory_path, file_name)
        try:
            process_jsonl(file_path, db_conn)
        except Exception as e:
            logging.error(f"Error processing file {file_name}: {e}")

## new SQL functions below ##
def process_all_prompts(conn):
    cursor = conn.cursor()
    try:
        # Fetch all prompts from the newspaper_data table
        cursor.execute('SELECT rowid, [Full Prompt] FROM newspaper_data')
        prompts = cursor.fetchall()
        
        for row_id, prompt in tqdm(prompts, desc="Processing prompts"):
            try:
                process_prompt(conn, row_id, prompt)
            except Exception as e:
                logging.error(f"Error processing prompt with row_id {row_id}: {e}")
        
        logging.info(f"Processed {len(prompts)} prompts from the database.")
    except sqlite3.Error as e:
        logging.error(f"Database error while fetching prompts: {e}")
def process_prompt(conn, row_id, prompt):
    try:
        data = json.loads(prompt)
        messages = data['body']['messages']
        custom_id = data.get('custom_id', f"row_{row_id}")
        
        cursor = conn.cursor()
        
        # Check if this custom_id has already been processed
        cursor.execute('SELECT id FROM completions WHERE custom_id = ?', (custom_id,))
        if cursor.fetchone():
            logging.info(f"Skipping already processed custom_id: {custom_id}")
            return
        
        completion = client.chat.completions.create(
            model='gpt-3.5-turbo',
            response_format={"type": "json_object"},
            messages=messages,
            max_tokens=data['body']['max_tokens']
        )
        
        json_response = completion.choices[0].message.content
        
        # Store the result in the completions table
        cursor.execute('INSERT INTO completions (custom_id, content) VALUES (?, ?)',
                       (custom_id, json_response))
        
        # Extract and store event data
        extract_and_store_event_data(cursor, custom_id, json_response)
        
        conn.commit()
        
    except json.JSONDecodeError:
        logging.error(f"Error decoding JSON for row_id {row_id}")
    except Exception as e:
        logging.error(f"Error during API call for row_id {row_id}: {e}")

def extract_and_store_event_data(cursor, custom_id, json_response):
    try:
        event_data = json.loads(json_response)
        cursor.execute('''
            INSERT OR REPLACE INTO events 
            (custom_id, konsert_datum, konsert_namn, lokal_namn, konserttyp_namn, producer)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            custom_id,
            event_data.get('konsert_datum', ''),
            event_data.get('konsert_namn', ''),
            event_data.get('lokal_namn', ''),
            event_data.get('konserttyp_namn', ''),
            event_data.get('Producer', '')
        ))
    except json.JSONDecodeError:
        logging.error(f"Error decoding JSON for custom_id: {custom_id}")
    except Exception as e:
        logging.error(f"Error storing event data for custom_id {custom_id}: {e}")



def create_db_tables(conn):
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS completions (
            id INTEGER PRIMARY KEY,
            custom_id TEXT UNIQUE,
            content TEXT
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY,
            custom_id TEXT UNIQUE,
            konsert_datum TEXT,
            konsert_namn TEXT,
            lokal_namn TEXT,
            konserttyp_namn TEXT,
            producer TEXT
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Results (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            prompt TEXT NOT NULL,
            result TEXT
        )
    ''')
    conn.commit()

def fetch_prompts_from_db(conn):
    """Fetch JSON prompts from the newspaper_data table."""
    fetch_sql = "SELECT [Full Prompt] FROM newspaper_data"
    try:
        cursor = conn.cursor()
        cursor.execute(fetch_sql)
        prompts = cursor.fetchall()
        logging.info(f"Fetched {len(prompts)} prompts from the database.")
        return [prompt[0] for prompt in prompts]
    except sqlite3.Error as e:
        logging.error(f"Failed to fetch prompts: {e}")
        return []

def save_results_to_db(conn, results):
    """Save results to the Results table in the database."""
    insert_sql = "INSERT INTO Results (prompt, result) VALUES (?, ?)"
    try:
        cursor = conn.cursor()
        cursor.executemany(insert_sql, results)
        conn.commit()
        logging.info(f"Saved {len(results)} results to the database.")
    except sqlite3.Error as e:
        logging.error(f"Failed to save results: {e}")


def main():
    # Load environment variables
    load_dotenv()

    # Set up logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Initialize OpenAI client
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

    # Database path
    db_path = 'Datasets/30.06.test.db'
    
    # Ensure database file exists
    if not os.path.exists(db_path):
        logging.error(f"Database file not found: {db_path}")
        return

    try:
        # Connect to the database
        with sqlite3.connect(db_path) as conn:
            # Create necessary tables
            create_db_tables(conn)
            
            # Process all prompts from the newspaper_data table
            process_all_prompts(conn)
            
            logging.info("Processing completed successfully.")
    
    except sqlite3.Error as e:
        logging.error(f"SQLite error: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
    finally:
        logging.info("Script execution completed.")

if __name__ == "__main__":
    main()

Processing prompts:   1%|          | 2/252 [00:04<08:14,  1.98s/it]2024-06-30 17:14:01,803 - INFO - Skipping already processed custom_id: dark-3693618-1-1-1
2024-06-30 17:14:01,804 - INFO - Skipping already processed custom_id: dark-3693618-1-1-1
2024-06-30 17:14:01,805 - INFO - Skipping already processed custom_id: dark-3693618-1-1-1
2024-06-30 17:14:03,404 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing prompts:   2%|▏         | 6/252 [00:05<03:07,  1.31it/s]2024-06-30 17:14:03,410 - INFO - Skipping already processed custom_id: dark-3693712-1-2-1
2024-06-30 17:14:03,411 - INFO - Skipping already processed custom_id: dark-3693712-1-2-1
2024-06-30 17:14:03,412 - INFO - Skipping already processed custom_id: dark-3693712-1-2-1
2024-06-30 17:14:05,465 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing prompts:   4%|▍         | 11/252 [00:09<03:22,  1.19it/s]2024-06-30 17:14:07,418 - INFO -

KeyboardInterrupt: 